In [60]:
import socialsent
from socialsent import util
import pyximport
pyximport.install()
from socialsent.representations import ppmigen, cooccurgen, makelowdim, sparse_io


In [61]:
def word_gen(file, gdict):
    for i, line in enumerate(open(file)):
        info = line.split("\t")
        comment = info[-1]
        for word in comment.split():
            word = word.lower()
#            if word != "<EOS>" and word in gdict.token2id:
            if word in gdict.token2id:
                yield word
        if i % 10000 == 0:
            print "Processed line", i

In [62]:
gdict = util.load_pickle("twitter_dict.pkl")

In [63]:
print gdict

Dictionary(0 unique tokens: [])


In [54]:
gdict.filter_extremes(no_above=0.5, no_below=50)
gdict.compactify()
outpath = "data/"
util.write_pickle(gdict.token2id, outpath + "index.pkl")

In [55]:
from collections import Counter
import os
import numpy as np
from socialsent import util

def run(word_gen, index, window_size, out_file):
    context = []
    pair_counts = Counter()
    for word in word_gen:
        context.append(index[word])
        if len(context) > window_size * 2 + 1:
            context.pop(0)
        pair_counts = _process_context(context, pair_counts, window_size)
    import pyximport
    pyximport.install(setup_args={"include_dirs": np.get_include()})
    sparse_io.export_mat_from_dict(pair_counts, out_file)

def _process_context(context, pair_counts, window_size):
    if len(context) < window_size + 1:
        return pair_counts
    target = context[window_size]
    indices = range(0, window_size)
    indices.extend(range(window_size + 1, 2 * window_size + 1))
    for i in indices:
        if i >= len(context):
            break
        pair_counts[(target, context[i])] += 1
    return pair_counts

In [56]:
run(word_gen("tweets_M.pkl", gdict), gdict.token2id, 4,  outpath + "counts.bin")


Processed line 0
Processed line 10000
Processed line 20000
Processed line 30000
Processed line 40000
Processed line 50000
Processed line 60000
Processed line 70000
Processed line 80000
Processed line 90000
Processed line 100000
Processed line 110000
Processed line 120000
Processed line 130000
Processed line 140000
Processed line 150000
Processed line 160000
Processed line 170000
Processed line 180000
Processed line 190000
Processed line 200000
Processed line 210000
Processed line 220000
Processed line 230000
Processed line 240000
Processed line 250000
Processed line 260000
Processed line 270000
Processed line 280000
Processed line 290000
Processed line 300000
Processed line 310000
Processed line 320000
Processed line 330000
Processed line 340000
Processed line 350000
Processed line 360000
Processed line 370000
Processed line 380000
Processed line 390000
Processed line 400000
Processed line 410000
Processed line 420000
Processed line 430000
Processed line 440000
Processed line 450000
Pr

In [57]:
from socialsent.representations.representation_factory import create_representation
from scipy.sparse import coo_matrix

def make_ppmi_mat(old_mat, row_probs, col_probs, smooth, neg=1, normalize=False):
    prob_norm = old_mat.sum() + (old_mat.shape[0] * old_mat.shape[1]) * smooth
    old_mat = old_mat.tocoo()
    row_d = old_mat.row
    col_d = old_mat.col
    data_d = old_mat.data
    neg = np.log(neg)
    for i in xrange(len(old_mat.data)):
        if data_d[i] == 0.0:
            continue
        joint_prob = (data_d[i] + smooth) / prob_norm
        denom = row_probs[row_d[i], 0] * col_probs[0, col_d[i]]
        if denom == 0.0:
            data_d[i] = 0
            continue
        data_d[i] = np.log(joint_prob /  denom)
        data_d[i] = max(data_d[i] - neg, 0)
        if normalize:
            data_d[i] /= -1*np.log(joint_prob)
    return coo_matrix((data_d, (row_d, col_d)))

def ppmi_run(count_path, out_path, smooth=0, cds=True, normalize=False, neg=1):
    counts = create_representation("Explicit", count_path, normalize=False)
    old_mat = counts.m
    index = counts.wi
    smooth = old_mat.sum() * smooth

    # getting marginal probs
    row_probs = old_mat.sum(1) + smooth
    col_probs = old_mat.sum(0) + smooth
    if cds:
        col_probs = np.power(col_probs, 0.75)
    row_probs = row_probs / row_probs.sum()
    col_probs = col_probs / col_probs.sum()

    # building PPMI matrix
    ppmi_mat = make_ppmi_mat(old_mat, row_probs, col_probs, smooth, neg=neg, normalize=normalize)
    import pyximport
    pyximport.install(setup_args={"include_dirs": np.get_include()})
    sparse_io.export_mat_eff(ppmi_mat.row, ppmi_mat.col, ppmi_mat.data, out_path + ".bin")
    util.write_pickle(index, out_path + "-index.pkl")

In [58]:
ppmi_run(outpath + "counts.bin", outpath + "ppmi", cds=True)

Could not find local index. Attempting to load directory wide index...


In [59]:
makelowdim.run(outpath + "ppmi.bin", outpath + "vecs")

'data/counts.bin'